In [10]:
import pandas as pd
import os
from nltk.tokenize import sent_tokenize
from typing import List

In [3]:
def delete_context(text: str) -> str:
    if text[0] == "[":
        n_strings = len(text)
        hook_end = 0
        while text[hook_end] != "]" and hook_end < 50:
            hook_end += 1
            if hook_end==(n_strings-1):
                return text

        if hook_end < 50:
            return text[(hook_end + 1) :].lstrip()
        else:
            return text.lstrip()

    else:
        return text.lstrip()

In [4]:
DATA_PATH = os.path.join(
    "..", "..", "..", "..", "data", "frameworks_data", "data_v0.7.1"
)

test_df = pd.read_csv(
    os.path.join(DATA_PATH, "new_columns_test_v0.7.1.csv.gz"), compression="gzip"
).drop_duplicates()
#test_df['target'] = test_df['target'].apply(literal_eval)

test_df = test_df[(test_df.lang=='en') & (test_df.target.apply(lambda x: len(x)>0))].drop(columns=['lang'])
test_df['excerpt'] = test_df['excerpt'].apply(delete_context)
test_df['excerpt'] = test_df['excerpt'].apply(sent_tokenize)
test_df = test_df.explode('excerpt')
test_df['len_excerpt'] = test_df['excerpt'].apply(lambda x: len(x.split(' ')))
test_df = test_df[test_df.len_excerpt<30]

# For LGBTQI + to become LGBTQI+
test_df['excerpt'] = test_df['excerpt'].apply(lambda x: x.replace(' +', '+'))

test_df.shape

(25028, 6)

In [5]:
months = ['january', 'february', 'march', 'april', 'june', 'july', 'august', 'september', 'october', 'november', 'december']

In [8]:
months_df = test_df.copy()

months_df['kw'] = months_df.excerpt.apply(
    lambda x:  ', '.join([one_month for one_month in months if one_month in x.lower()])
)
months_df = months_df[months_df['kw'].apply(lambda x: ',' not in x and len(x)>0)]


In [9]:
months_df

,excerpt,entry_id,target,analysis_framework_id,project_id,len_excerpt,kw
9,The number of confirmed COVID-19 cases reached...,187035.0,"['first_level_tags->sectors->Cross', 'first_le...",829.0,1184.0,17,august
9,• As the COVID-19 cases relentlessly surge at ...,187035.0,"['first_level_tags->sectors->Cross', 'first_le...",829.0,1184.0,24,august
9,"Consequently, the national health emergency go...",187035.0,"['first_level_tags->sectors->Cross', 'first_le...",829.0,1184.0,10,september
98,Both governments moved by the end of March to ...,186821.0,"['first_level_tags->sectors->Livelihoods', 'fi...",829.0,1184.0,16,march
111,"A total of 1,473 movements were recorded, comp...",187325.0,"['first_level_tags->pillars_1d->Displacement',...",1306.0,2170.0,19,november
...,...,...,...,...,...,...,...
23602,On 11 December another attack involving EEI re...,304924.0,"['first_level_tags->sectors->Protection', 'fir...",1465.0,2334.0,20,december
23604,The border was partially reopened for trucks c...,323784.0,"['first_level_tags->sectors->Cross', 'first_le...",1465.0,2334.0,15,march
23623,The API NGO distributing food rations on behal...,310331.0,"['first_level_tags->sectors->Food Security', '...",1465.0,2333.0,29,june
23660,• In the month of August unlike the general dy...,188418.0,[],829.0,1183.0,26,august


In [11]:
def get_augmented_df(original_df, kw: List[str], name: str):

    df = original_df.copy()
    df['kw'] = df['excerpt'].apply(
        lambda x: [word for word in kw if word in x.lower()]
    )
    df = df[df.kw.apply(lambda x: len(x)==1)]
    df['kw'] = df['kw'].apply(lambda x: x[0])
    df['type'] = 'original'
    print(len(df))

    final_df = pd.DataFrame()
    for i in range(df.shape[0]):
        row = df.iloc[[i]]
        original_row_kw = row["kw"].values[0]
        row["type"] = "augmented"
        for one_kw in kw:
            if one_kw != original_row_kw:
                row_tmp = row.copy()
                row_tmp["excerpt"] = row_tmp["excerpt"].apply(
                    lambda x: x.replace(original_row_kw, one_kw).replace(
                        original_row_kw.capitalize(), one_kw.capitalize()
                    )
                )
                row_tmp["kw"] = [one_kw]
                final_df = final_df.append(row_tmp)

    final_df = pd.concat(
        [df, final_df]
    )

    final_df.to_csv(f'final_data/{name}.csv', index=None)

    return final_df

In [12]:
get_augmented_df(
    months_df[['entry_id', 'excerpt']],
    kw=months,
    name='months'
)

1919


<ipython-input-11-e9c0b3c9eb12>:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  row["type"] = "augmented"


,entry_id,excerpt,kw,type
9,187035.0,The number of confirmed COVID-19 cases reached...,august,original
9,187035.0,• As the COVID-19 cases relentlessly surge at ...,august,original
9,187035.0,"Consequently, the national health emergency go...",september,original
98,186821.0,Both governments moved by the end of March to ...,march,original
111,187325.0,"A total of 1,473 movements were recorded, comp...",november,original
...,...,...,...,...
23671,209380.0,August 2015 armed groups extend their attacks ...,august,augmented
23671,209380.0,September 2015 armed groups extend their attac...,september,augmented
23671,209380.0,October 2015 armed groups extend their attacks...,october,augmented
23671,209380.0,November 2015 armed groups extend their attack...,november,augmented
